In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize, signal

In [2]:
df = pd.read_csv("psf_magnitudes.csv")
df = df.drop(columns = ['Unnamed: 0'])
df.head()

,time,tar_mag_psf,tar_mag_psf_err,filter
0,61370.269,14.405148,0.147069,z
1,62278.228,14.373457,0.232219,z
2,62837.873,14.427374,0.365593,z
3,63739.645,14.549014,0.323183,z
4,66798.808,14.322212,0.098737,z


In [3]:
def sin_harm(x, T, a0, a1, phi1):
    # a1 = 0.8
    # a2 = 0.8
    # # phi2 = phi1
    w = 2 * np.pi / (3600*T)
    return a0 + a1 * np.sin(w * x + phi1)

def mse(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

In [4]:
def sin_harm_2nd(x, T, a0, a1, phi1, a2, phi2):
        w = 2 * np.pi / (3600*T)
        
        # if k==0:
        #     a0 = 14.3
        #     # w2 = 0.5
        # elif k==1:
        #     a0 = 14.3
        #     # w2 = 0.32
        # elif k==2:
        #     a0 = 14.3
        #     # w2 = 0.28
        # elif k==3:
        #     a0 = 14.3
            # w2 = 0.29
        # a0 = (max(y_data[band])+min(magnitudes))/2
        # return signal.sawtooth(w1 * x + phi1, w2) * a1 + a0
        return a0 + a1 * np.sin(w * x + phi1) + a2 * np.sin( 2 * w * x + phi2)

In [5]:
filters = df['filter'].unique()
filters

array(['z', 'i', 'g', 'r'], dtype=object)

In [6]:
params = {}
params_covariance = {}
for filter in filters:
    time_series = df[(df['filter']==filter)]['time']
    magnitudes = df[(df['filter']==filter)]['tar_mag_psf']
    # print(time_series, magnitudes)
    params[filter], params_covariance[filter] = optimize.curve_fit(sin_harm_2nd, time_series, magnitudes,
                                                                   bounds = ([3]+[-np.inf]*5,
                                                                             [np.inf]*6))

    y_fit1 = sin_harm_2nd(np.arange(min(time_series), max(time_series), 200), *params[filter])
    y_fit2 = sin_harm_2nd(time_series, *params[filter])

    plt.gca().invert_yaxis() 
    plt.plot(time_series, magnitudes, label='data')
    plt.plot(np.arange(min(time_series), max(time_series), 200), y_fit1, label='fit')
    plt.plot(time_series, y_fit2, label='fit (only at data points)')
    plt.legend(loc='best')

    plt.title('band: {}, time: {:.2f}h, fit mse error: {:.5f}'.format(filter,params[filter][0], mse(magnitudes, y_fit2)))
    plt.savefig(filter+'.png')
    plt.close()
print(params)


{'z': array([ 5.27485743, 14.12164428,  0.3398218 ,  5.80056177,  0.11645256,
       15.12503629]), 'i': array([ 4.23773266, 13.75120682,  0.36787977,  1.86780022,  0.71899974,
        6.31711522]), 'g': array([ 5.38192408, 14.14360215,  0.56020006,  6.39937256,  0.19220261,
       16.44777329]), 'r': array([ 4.26126061, 14.00701998,  0.42239319,  0.58491316,  0.13125554,
       -5.9422181 ])}


In [7]:
final_params = {}
final_params_covariance = {}

final_periods = {}

for filter in filters:
    error = np.inf
    time_series = df[(df['filter']==filter)]['time']
    magnitudes = df[(df['filter']==filter)]['tar_mag_psf']

    for time_period in np.arange(3,24,0.2):
        params, params_covariance = optimize.curve_fit((lambda x, a0, a1, phi1, a2, phi2 : sin_harm_2nd(x, time_period, a0, a1, phi1, a2, phi2)),
                                                                    time_series, magnitudes)

        y_fit1 = sin_harm_2nd(np.arange(min(time_series), max(time_series), 200), time_period, *params)
        y_fit2 = sin_harm_2nd(time_series, time_period, *params)

        this_error = mse(magnitudes, y_fit2)
        if (error > this_error):
            error = this_error
            final_periods[filter] = time_period
            final_params[filter] = params
            final_params_covariance[filter] = params_covariance

    y_fit1 = sin_harm_2nd(np.arange(min(time_series), max(time_series), 200), final_periods[filter], *final_params[filter])
    y_fit2 = sin_harm_2nd(time_series, final_periods[filter], *final_params[filter])

    plt.gca().invert_yaxis() 
    plt.plot(time_series, magnitudes, label='data')
    plt.plot(np.arange(min(time_series), max(time_series), 200), y_fit1, label='fit')
    plt.plot(time_series, y_fit2, label='fit (only at data points)')
    plt.legend(loc='best')

    plt.title('band: {}, period: {:.2f}h, fit mse error: {:.5f}'.format(filter,final_periods[filter], error))
    plt.savefig(filter+'.png')
    plt.close()

print(final_periods, final_params)


{'z': 5.200000000000002, 'i': 6.200000000000003, 'g': 5.400000000000002, 'r': 5.8000000000000025} {'z': array([14.122185  ,  0.34447141,  5.48043327,  0.11236247,  1.92489632]), 'i': array([14.10708421, -0.32229425, -0.0975062 , -0.16244216,  0.39291146]), 'g': array([14.14457711,  0.55999861,  0.19332247, -0.19025368,  0.88554259]), 'r': array([14.06597869,  0.41871422,  1.63722088,  0.16093129,  0.67273845])}


In [11]:
final_params = {}
final_params_covariance = {}

final_periods = {}

for filter in filters:
    error = np.inf
    time_series = df[(df['filter']==filter)]['time']
    magnitudes = df[(df['filter']==filter)]['tar_mag_psf']

    for time_period in np.arange(3,24,0.2):
        params, params_covariance = optimize.curve_fit((lambda x, a1, phi1, a2, phi2 : sin_harm_2nd(x, time_period, 14.3, a1, phi1, a2, phi2)),
                                                                    time_series, magnitudes)

        y_fit1 = sin_harm_2nd(np.arange(min(time_series), max(time_series), 200), time_period, 14.3, *params)
        y_fit2 = sin_harm_2nd(time_series, time_period, 14.3, *params)

        this_error = mse(magnitudes, y_fit2)
        if (error > this_error):
            error = this_error
            final_periods[filter] = time_period
            final_params[filter] = params
            final_params_covariance[filter] = params_covariance

    y_fit1 = sin_harm_2nd(np.arange(min(time_series), max(time_series), 200), final_periods[filter], 14.3, *final_params[filter])
    y_fit2 = sin_harm_2nd(time_series, final_periods[filter], 14.3, *final_params[filter])

    plt.gca().invert_yaxis() 
    plt.plot(time_series, magnitudes, label='data')
    plt.plot(np.arange(min(time_series), max(time_series), 200), y_fit1, label='fit')
    plt.plot(time_series, y_fit2, label='fit (only at data points)')
    plt.legend(loc='best')

    plt.title('band: {}, period: {:.2f}h, fit mse error: {:.5f}'.format(filter,final_periods[filter], error))
    plt.savefig(filter+'.png')
    plt.close()

print(final_periods, final_params)


{'z': 11.000000000000007, 'i': 10.600000000000007, 'g': 7.800000000000004, 'r': 10.200000000000006} {'z': array([0.29691732, 0.58126495, 0.44434969, 0.19143381]), 'i': array([-0.45132829,  1.8684654 ,  0.12822582,  0.86554178]), 'g': array([ 0.52861268,  1.00628116, -0.21179187,  1.25401639]), 'r': array([-0.49157262,  1.48437779, -0.14734272,  8.66455048])}
